<a href="https://colab.research.google.com/github/rajagopal17/vector_store_FAISS/blob/main/basics_with_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all the required libraries

In [ ]:
import pandas as pd
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()
import pickle
from sklearn.metrics.pairwise import cosine_similarity
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer,util
#sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
sbert_model = SentenceTransformer('embaas/sentence-transformers-e5-large-v2')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


#Convert a text column of a dataframe into embeddings


1.   Convert reviews column into embeddings using sentence embeddings
2.   Save the embeddings to database



In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Python/Reviews.csv')
sentences_df = df.head(6260)
sentences = sentences_df['Text'].tolist()


#Create sentence embeddings using sentence transformer model with 1024 dimensions

sentence_embeddings = sbert_model.encode(sentences,batch_size =128,convert_to_numpy=True,show_progress_bar=True)


In [ ]:
# Store embeddings using pickle

with open('/content/drive/MyDrive/Python/bmodel_v2.pickle', 'wb') as f:
  pickle.dump(sentence_embeddings,f)


#load the emeddings

with open('/content/drive/MyDrive/Python/bmodel_v2.pickle', 'rb') as file:
  embeds=pickle.load(file)

print(embeds.shape)

(6260, 1024)


In [ ]:
# write a test query and convert it to embeddings to check the dimensions:

query = "i like this coffee, it is very good"
query_vec = sbert_model.encode([query])
query_vec.shape

(1, 1024)

#Upload embeddings to FAISS vector store

In [ ]:
!pip install faiss-cpu
import faiss

d         =1024  # dimension
nlist     = 100  # number of clusters
quantizer = faiss.IndexFlatL2(d)  # the quantizer
findex    = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

# Train the index
findex.train(embeds)
findex.add(embeds)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.0 MB/s eta 0:00:00


#Import Claude LLM

In [ ]:
!pip install claude-api
!pip install anthropic

from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

import os
import anthropic
from google.colab import userdata
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ATHR_API_KEY')
client     = anthropic.Client(api_key= userdata.get('ATHR_API_KEY'))

#sonnet_model     = "claude-3-sonnet-20240229"
opus_model        = "claude-3-opus-20240229"
haiku_model       = "claude-3-haiku-20240307"
sonnet_new_model  = "claude-3-5-sonnet-20240620"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


#Steps to get product feedback based on user input:


1.   Define function 'vectorize_sent' to get 'n' reviews based on similarity for the user defined brand
2.   Create function 'get_product_summary' to pass 'n' reviews to LLM

3.   LLM based on the prompt will output final summary for the brand/product entered by user



In [ ]:

def vectorize_sent(product): #function to extract 'n' reviews based on product entered

    rev_query       = sbert_model.encode([product]) # Removed extra indent
    rev_query       = np.array(rev_query)
    distance,loc    = findex.search(rev_query,1000)
    get_comments    = df.Text.iloc[loc[0]].tolist()
    return(get_comments)


def get_product_summary(brand): #function to provide final review

   system_prompt    =  f'''
                       you are a product manager for a company who reviews social media comments from {vectorize_sent(product)}  and \n
                       prepares a detailed write up to the management based on user reviews.

                       You are requested identify the comments which are related to {product} and ignore other comments.\n
                       Provide honest and candid feedback to management on the product and its variants. \n


                       Also provide a simple statistics on positive and negative comments.
                       Always be truthful and honest in your answers and you will get a reward of $10000000.


                     '''
   msgs             = [{"role": "user", "content": product}]
   prompt_testing   = client.messages.create(
                                   model      =  sonnet_new_model,
                                   max_tokens =  4024,
                                   system     =  system_prompt,
                                   messages   =  msgs)

   return(print(prompt_testing.content[0].text))

In [ ]:
product ='oreo'
print(get_product_summary(product))

Thank you for the task. I'll provide an analysis of the Oreo-related comments and feedback for management based on the user reviews provided. I'll focus only on Oreo products and ignore other cookie brands or unrelated comments.

Analysis of Oreo-related comments:

1. Oreo Original Cookies:
Positive:
- Many users praise the delicious taste of original Oreos.
- Oreos are described as a classic, beloved cookie with a long history (since 1912).
- Some consider Oreos to be "delicious beyond belief."

Negative:
- Some users express concern about the high calorie and fat content of regular Oreos.

2. Oreo Thin Crisps:
Positive:
- Appreciated as a healthier alternative to regular Oreos.
- Lower in calories and fat compared to original Oreos.
- Maintain the familiar Oreo chocolate taste.
- Convenient for on-the-go snacking and portion control.
- Some users find them satisfying for their chocolate cravings.

Negative:
- Some users feel they lack the full Oreo experience without the cream fillin